<a href="https://colab.research.google.com/github/hafizhry/NER-resume/blob/main/Resume_Projects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy==2.1.4

     |████████████████████████████████| 29.8 MB 39 kB/s 
     |████████████████████████████████| 2.1 MB 56.4 MB/s 
     |████████████████████████████████| 82 kB 404 kB/s 
     |████████████████████████████████| 3.2 MB 40.5 MB/s 
  Attempting uninstall: preshed
    Found existing installation: preshed 3.0.5
    Uninstalling preshed-3.0.5:
      Successfully uninstalled preshed-3.0.5
  Attempting uninstall: plac
    Found existing installation: plac 1.1.3
    Uninstalling plac-1.1.3:
      Successfully uninstalled plac-1.1.3
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninstalled blis-0.4.1
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
ERROR: pip's dependency resolve

In [ ]:
!pip install PyMuPDF

     |████████████████████████████████| 6.4 MB 7.9 MB/s 


In [ ]:
import spacy
import pickle 
import random
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
from spacy.lang.en import English

In [ ]:
import json 
def open_dataset(file_path):
  dataset = []
  lines = []
  with open(file_path,) as f:
    lines = f.readlines() 
    print('Number of data: ' + str(len(lines)))
    for line in lines:
      data = json.loads(line)
      text = data['content']
      entities = []
      for annotation in data['annotation']:
        point = annotation['points'][0]
        labels = annotation['label']
        if not isinstance(labels, list):
          labels = list(labels)
        for label in labels:
          entities.append((point['start'], point['end']+1, label))
      
      dataset.append((text, {'entities':entities}))
  return dataset

In [ ]:
data = open_dataset('/content/Entity Recognition in Resumes.json')

Number of data: 220


In [ ]:
print(data[0])
print(len(data))

("Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\n• To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya Vidyalaya\n

In [ ]:
data[0][0][21]

'i'

In [ ]:
import re
def cleaning_dataset(data):
  invalid_span_tokens = re.compile(r'\s')
  cleaned_data = []
  for text, annotations in data:
      entities = annotations['entities']
      valid_entities = []
      cleaned_text = ' '.join(text.split('\n'))
      cleaned_text = cleaned_text.lower()
      for start, end, label in entities:
          valid_start = start
          valid_end = end
          while valid_start < len(text) and invalid_span_tokens.match(
                  text[valid_start]):
              valid_start += 1
          while valid_end > 1 and invalid_span_tokens.match(
                  text[valid_end - 1]):
              valid_end -= 1
          valid_entities.append([valid_start, valid_end, label])
      cleaned_data.append([cleaned_text, {'entities': valid_entities}])
  return cleaned_data

In [ ]:
cleaned_data = cleaning_dataset(data)

In [ ]:
cleaned_data

[["puran mal jaipur, rajasthan - email me on indeed: indeed.com/r/puran-mal/357ea77b3b002be6  work experience  admin assistant at infosys limited  front office work  education  bachelor's  skills  front office executive (1 year)  additional information  technical skill • basic knowledge of computer operating • ms office (ms excel, ms word and power point) • with typing speed 25 wpm personal skill • positive attitude • good decision making • good communication skills • confident • have zeal to learn new things hobby • listening music • traveling  https://www.indeed.com/r/puran-mal/357ea77b3b002be6?isid=rex-download&ikw=download-top&co=in",
  {'entities': [[194, 511, 'Skills'],
    [174, 183, 'Degree'],
    [127, 142, 'Companies worked at'],
    [108, 123, 'Designation'],
    [50, 89, 'Email Address'],
    [10, 16, 'Location'],
    [0, 9, 'Name']]}],
 ['sridevi h bangalore, karnataka - email me on indeed: indeed.com/r/sridevi-h/63703b24aaaa54e4  to further my career with a growth-oriente

In [ ]:
def train_data_spacy(train_data):
  nlp = English()
  if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
  else: 
    ner = nlp.get_pipe('ner')

  for _, annotation in train_data:
      for annot in annotation['entities']:
        ner.add_label(annot[2])
  
  other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
  with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    n_iter = 50
    
    for itn in range(n_iter):
      print('Iteration ' + str(itn))
      random.shuffle(train_data)
      losses = {}
      for batch in spacy.util.minibatch(train_data, size=2):
        texts = [text for text, annotation in batch]
        annotations = [annotation for text, annotation in batch] 
        try:
          nlp.update(
                texts, 
                annotations,
                drop=0.3,
                sgd=optimizer,
                losses=losses)
        except Exception as e:
          pass
      print("Losses", losses)
    
  nlp.to_disk('models')

In [ ]:
train_data_spacy(data)

Iteration 0
Losses {'ner': 22776.750091799913}
Iteration 1
Losses {'ner': 14628.398484686919}
Iteration 2
Losses {'ner': 14782.772316128929}
Iteration 3
Losses {'ner': 14356.039938477199}
Iteration 4
Losses {'ner': 14330.210438321865}
Iteration 5
Losses {'ner': 12185.176002353277}
Iteration 6
Losses {'ner': 13085.276681874113}
Iteration 7
Losses {'ner': 11004.931413100765}
Iteration 8
Losses {'ner': 10526.294927726496}
Iteration 9
Losses {'ner': 10177.605257141166}
Iteration 10
Losses {'ner': 9927.00980228683}
Iteration 11
Losses {'ner': 9899.207335380093}
Iteration 12
Losses {'ner': 8686.423027323208}
Iteration 13
Losses {'ner': 7666.154590589537}
Iteration 14
Losses {'ner': 9461.886054081753}
Iteration 15
Losses {'ner': 8312.762285376908}
Iteration 16
Losses {'ner': 7177.267999038841}
Iteration 17
Losses {'ner': 7940.191388379464}
Iteration 18
Losses {'ner': 7035.645297745981}
Iteration 19
Losses {'ner': 6332.462632563063}
Iteration 20
Losses {'ner': 7874.5780252815575}
Iteration 21


In [ ]:
import sys, fitz
import os
def upload_resume():
  base_dir = '/content'
  from google.colab import files
  uploaded = files.upload()
  for fn in uploaded.keys():
    print(os.path.join(base_dir, fn))
    doc = fitz.open(os.path.join(base_dir, fn))
  text = ''
  for page in doc:
    text = text + str(page.getText())
    tx = ' '.join(text.split('\n'))
    #tx = tx.lower()
  #print(tx)

  nlp_model = spacy.load('models')
  test = nlp_model(tx)
  for ent in test.ents:
    print(f'{ent.label_.upper():{20}}- {ent.text}')

In [ ]:
upload_resume()